# Python for Survey Data Preparation into SPSS

In [1]:
import pandas as pd
import numpy as np

# import fungsi-fungsi untuk melakukan remap
import reassign_into_spss as rps 

In [2]:
# Optional, untuk menampilkan seluruh kolom dataset

pd.set_option("display.max_columns",None)

# Import data

## Dataset

Dataset yang digunakan pada contoh ini merupakan dataset sederhana yang dibuat khusus untuk mengilustrasikan keseluruhan tahapan proses pada contoh ini.

In [3]:
dataset_ = pd.read_excel("dataset/dataset.xlsx", engine='openpyxl', header=[0,1])

In [4]:
dataset = dataset_.copy()
dataset.head()

nomor                 id      alamat    kota Apakah puas?  \
  Unnamed: 0_level_1 Unnamed: 1_level_1 kode alamat jawaban   q1 jawaban   
0                  1             100111       aaaaa       1            2   
1                  2             100112       bbbbb       2            2   
2                  3             100113       ccccc       3            1   
3                  4             100114       ddddd       2            2   
4                  5             100115       eeeee       2            1   

   Alasan puas                           Alasan utama kenyamanan  \
  q2 jawaban 1 q2 jawaban 2 q2 jawaban 3              q3 jawaban   
0          NaN          NaN          NaN                     NaN   
1          NaN          NaN          NaN                     NaN   
2          3.0          NaN          NaN                     3.0   
3          NaN          NaN          NaN                     NaN   
4          2.0          3.0          NaN                     2.0   

  Tempat membeli                                                      \
    q4 jawaban 1 q4 jawaban 2 q4 jawaban 3 q4 jawaban 4 q4 jawaban 5   
0              2          NaN          NaN          NaN          NaN   
1              1          2.0          4.0          5.0          NaN   
2              3         10.0          NaN          NaN          NaN   
3              3          NaN          NaN          NaN          NaN   
4              3          4.0          5.0          NaN          NaN   

                                                                     \
  q4 jawaban 6 q4 jawaban 7 q4 jawaban 8 q4 jawaban 9 q4 jawaban 10   
0          NaN          NaN          NaN          NaN           NaN   
1          NaN          NaN          NaN          NaN           NaN   
2          NaN          NaN          NaN          NaN           NaN   
3          NaN          NaN          NaN          NaN           NaN   
4          NaN          NaN          NaN          NaN           NaN   

  Tempat paling sering beli Frekuensi beli Harga terakhir satuan pembelian   \
                 q5 jawaban     q6 jawaban                       q7 jawaban   
0                         2              4                             5000   
1                         1              2                             7000   
2                         3             10                             6500   
3                         3              3                             6000   
4                         4              9                             5000   

  Tingkat kemurahan Kata kesan Rekomendasi    Website favorit  
         q7 jawaban q8 jawaban  q9 jawaban    website favorit  
0                 3          2           2     www.google.com  
1                 1          1           2    www.youtube.com  
2                 2          4           1  www.wikipedia.org  
3                 2          3           2    www.youtube.com  
4                 4          5           1    www.youtube.com

## Daftar pertanyaan

Daftar pertanyaan merupakan file yang dibuat untuk panduan penggantian kolom dataset yang semula memiliki dua header menjadi dataset dengan satu header saja. 

- Header lama --> [raw pertanyaan, raw subpertanyaan]
- Header baru --> [new subpertanyaan]


Kolom <b>label pertanyaan</b> digunakan sebagai penanda kolom dataset termasuk ke dalam label pertanyaan apa. Selain itu, <b>label pertanyaan</b> berfungsi sebagai pembeda antara kolom-kolom informasi responden dan kolom pertanyaan kuesioner.

- Kolom informasi responden : memiliki nilai "NaN" pada kolom <b>label pertanyaan</b> (pada kasus ini di atas "kota")
- Kolom pertanyaan kuesioner : merupakan kolom yang berisi jawaban-jawaban responden terhadap pertanyaan kuesioner

Untuk pembuatan lebih jelasnya, dapat langsung membuka file excel yang tersedia.

<br>
Note : tahapan  ini (2.2) merupakan implementasi dari <a href="https://youtu.be/pKvWD0f18Pc">https://youtu.be/pKvWD0f18Pc</a>


In [5]:
daftar_pertanyaan_ = pd.read_excel("dataset/daftar pertanyaan.xlsx",  engine='openpyxl')

In [6]:
daftar_pertanyaan = daftar_pertanyaan_.copy()
daftar_pertanyaan.head(10)

,raw pertanyaan,raw subpertanyaan,new pertanyaan,new subpertanyaan,label pertanyaan
0,nomor,NaN,nomor,nomor,NaN
1,id,NaN,id,id,NaN
2,alamat,kode alamat,alamat,kode_alamat,NaN
3,kota,jawaban,kota,kota,kota
4,Apakah puas?,q1 jawaban,Apakah puas?,q1_puas,q1
5,Alasan puas,q2 jawaban 1,Alasan puas,q2_alsn_puas_1,q2
6,NaN,q2 jawaban 2,Alasan puas,q2_alsn_puas_2,q2
7,NaN,q2 jawaban 3,Alasan puas,q2_alsn_puas_3,q2
8,Alasan utama kenyamanan,q3 jawaban,Alasan utama kenyamanan,q3_alsn_puas_utm,q3
9,Tempat membeli,q4 jawaban 1,Tempat membeli,q4_tmpt_bli_1,q4


## Label jawaban

Label jawaban berfungsi sebagai keterangan nilai pada masing-masing jawaban di kolom dataset. Header yang digunakan pada kasus ini adalah label pertanyaan. Kolom dataset dengan label pertanyaan yang merupakan data kategori akan memiliki nilai selain "NaN" pada file ini, sedangkan kolom dataset yang merupakan data non-kategori akan memiliki nilai "NaN" seluruhnya.

<b>label_jawaban</b> dimulai dengan 0 untuk menyesuaikan format penomoran indeks pertama pada Python.

<br>
<i> Note : contoh penerapan ini menganggap semua jawaban yang aslinya bernilai "99, lainnya ..." sudah di-handle menjadi nilai lain ke dalam kolom <b>label_jawaban</b> </i>

In [7]:
label_jawaban_ = pd.read_excel("dataset/label jawaban.xlsx", engine='openpyxl')

In [8]:
label_jawaban = label_jawaban_.copy()
label_jawaban

,label_jawaban,kota,q1,q2,q3,q4,q5,q6,q7,q8,q9,web
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Jakarta,Ya,Kualitas baik,Mudah menggunakannya,Supermarket,Sulit digunakan,NaN,Sangat Mahal,Sangat jelek,Ya,NaN
2,2,Aceh,Tidak,Harganya murah,Harganya murah,Pasar tradisional,Harga kurang terjangkau,NaN,Mahal,Jelek,Tidak,NaN
3,3,Palembang,NaN,Dapat ditemui di mana saja,Dapat ditemui di mana saja,Warung,Tidak banyak dijumpai,NaN,Biasa saja,Biasa saja,NaN,NaN
4,4,NaN,NaN,NaN,NaN,Online shop,NaN,NaN,Murah,Bagus,NaN,NaN
5,5,NaN,NaN,NaN,NaN,Dari keluarga,NaN,NaN,Sangat murah,Sangat bagus,NaN,NaN
6,6,NaN,NaN,NaN,NaN,Dari teman,NaN,NaN,NaN,NaN,NaN,NaN
7,7,NaN,NaN,NaN,NaN,Mini market,NaN,NaN,NaN,NaN,NaN,NaN
8,8,NaN,NaN,NaN,NaN,Pom bensin,NaN,NaN,NaN,NaN,NaN,NaN
9,9,NaN,NaN,NaN,NaN,Toko sembako,NaN,NaN,NaN,NaN,NaN,NaN


# Dataset 1 header

Melakukan penggantian nama kolom dataset

In [9]:
dataset_1h = dataset.copy()

# Melakukan penggantian nama kolom
dataset_1h.columns = daftar_pertanyaan['new subpertanyaan'].to_list()


dataset_1h.head()

,nomor,id,kode_alamat,kota,q1_puas,q2_alsn_puas_1,q2_alsn_puas_2,q2_alsn_puas_3,q3_alsn_puas_utm,q4_tmpt_bli_1,q4_tmpt_bli_2,q4_tmpt_bli_3,q4_tmpt_bli_4,q4_tmpt_bli_5,q4_tmpt_bli_6,q4_tmpt_bli_7,q4_tmpt_bli_8,q4_tmpt_bli_9,q4_tmpt_bli_10,q5_tmpt_utm,q6_frek_bli,q7_harga_bli_rp,q7_kemurahan,q8_kesan,q9_rekomen,web_fav
0,1,100111,aaaaa,1,2,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,4,5000,3,2,2,www.google.com
1,2,100112,bbbbb,2,2,NaN,NaN,NaN,NaN,1,2.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1,2,7000,1,1,2,www.youtube.com
2,3,100113,ccccc,3,1,3.0,NaN,NaN,3.0,3,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,10,6500,2,4,1,www.wikipedia.org
3,4,100114,ddddd,2,2,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,6000,2,3,2,www.youtube.com
4,5,100115,eeeee,2,1,2.0,3.0,NaN,2.0,3,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,9,5000,4,5,1,www.youtube.com


Mengecek kolom-kolom baru dataset

In [10]:
# cek kolom-kolom informasi responden
print(dataset_1h.columns[:3])

print("")

# cek kolom-kolom jawaban pertanyaan kuesioner
print(dataset_1h.columns[3:])

Index(['nomor', 'id', 'kode_alamat'], dtype='object')

Index(['kota', 'q1_puas', 'q2_alsn_puas_1', 'q2_alsn_puas_2', 'q2_alsn_puas_3',
       'q3_alsn_puas_utm', 'q4_tmpt_bli_1', 'q4_tmpt_bli_2', 'q4_tmpt_bli_3',
       'q4_tmpt_bli_4', 'q4_tmpt_bli_5', 'q4_tmpt_bli_6', 'q4_tmpt_bli_7',
       'q4_tmpt_bli_8', 'q4_tmpt_bli_9', 'q4_tmpt_bli_10', 'q5_tmpt_utm',
       'q6_frek_bli', 'q7_harga_bli_rp', 'q7_kemurahan', 'q8_kesan',
       'q9_rekomen', 'web_fav'],
      dtype='object')


Pada kasus ini, kolom <b>web_fav</b> akan dipindahkan di antara <b>kode_alamat</b> dan <b>kota</b> serta akan dianggap sebagai salah satu kolom informasi responden.

In [11]:
# daftar urutan nama kolom yang baru
kolom_reindex = dataset_1h.columns[:3].to_list() + [dataset_1h.columns[-1]] + dataset_1h.columns[3:-1].to_list()

# mengurutkan kolom yang ada sesuai urutan yang baru
dataset_1h = dataset_1h.reindex(columns=kolom_reindex)

dataset_1h.head()

,nomor,id,kode_alamat,web_fav,kota,q1_puas,q2_alsn_puas_1,q2_alsn_puas_2,q2_alsn_puas_3,q3_alsn_puas_utm,q4_tmpt_bli_1,q4_tmpt_bli_2,q4_tmpt_bli_3,q4_tmpt_bli_4,q4_tmpt_bli_5,q4_tmpt_bli_6,q4_tmpt_bli_7,q4_tmpt_bli_8,q4_tmpt_bli_9,q4_tmpt_bli_10,q5_tmpt_utm,q6_frek_bli,q7_harga_bli_rp,q7_kemurahan,q8_kesan,q9_rekomen
0,1,100111,aaaaa,www.google.com,1,2,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,4,5000,3,2,2
1,2,100112,bbbbb,www.youtube.com,2,2,NaN,NaN,NaN,NaN,1,2.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1,2,7000,1,1,2
2,3,100113,ccccc,www.wikipedia.org,3,1,3.0,NaN,NaN,3.0,3,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,10,6500,2,4,1
3,4,100114,ddddd,www.youtube.com,2,2,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,6000,2,3,2
4,5,100115,eeeee,www.youtube.com,2,1,2.0,3.0,NaN,2.0,3,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,9,5000,4,5,1


# Re-assign ulang jawaban multiresponse

Jawaban pada <b>q2</b> disusun berdasarkan urutan responden menjawab pertama hingga terakhir. Nama kolom di q2 dimaksudkan : q2 alasan puas pertama, q2 alasan puas kedua, dst. Jika <b>q2</b> memiliki 5 pilihan jawaban, tetapi responden hanya menjawab 2 pilihan saja (apapun pilihannya), maka alasan puas ketiga, keempat, dan kelima akan "NaN". 

Terkadang, kita membutuhkan data yang bukan berformat seperti itu. Misal, kita ingin memiliki data dengan format kolom yang merepresentasikan opsi yang kita miliki. Contoh : kolom q2 alasan puas opsi jawaban 1 akan tidak bernilai "NaN" untuk responden yang menjawab <b>1</b>, kolom q2 alasan puas opsi jawaban 2 akan tidak bernilai "NaN" untuk responden yang menjawab <b>2</b>.

Oleh karena itu, kita perlu melakukan <i>assign</i> ulang jawaban yang sudah ada ke kolom-kolom yang kita inginkan.

Untuk lebih jelasnya, dapat dilihat pada ilustrasi contoh selanjutnya.

In [12]:
print("data tanpa label sebelum dilakukan re-assign value")
dataset_1h

data tanpa label sebelum dilakukan re-assign value


,nomor,id,kode_alamat,web_fav,kota,q1_puas,q2_alsn_puas_1,q2_alsn_puas_2,q2_alsn_puas_3,q3_alsn_puas_utm,q4_tmpt_bli_1,q4_tmpt_bli_2,q4_tmpt_bli_3,q4_tmpt_bli_4,q4_tmpt_bli_5,q4_tmpt_bli_6,q4_tmpt_bli_7,q4_tmpt_bli_8,q4_tmpt_bli_9,q4_tmpt_bli_10,q5_tmpt_utm,q6_frek_bli,q7_harga_bli_rp,q7_kemurahan,q8_kesan,q9_rekomen
0,1,100111,aaaaa,www.google.com,1,2,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,4,5000,3,2,2
1,2,100112,bbbbb,www.youtube.com,2,2,NaN,NaN,NaN,NaN,1,2.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1,2,7000,1,1,2
2,3,100113,ccccc,www.wikipedia.org,3,1,3.0,NaN,NaN,3.0,3,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,10,6500,2,4,1
3,4,100114,ddddd,www.youtube.com,2,2,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,6000,2,3,2
4,5,100115,eeeee,www.youtube.com,2,1,2.0,3.0,NaN,2.0,3,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,9,5000,4,5,1
5,6,100116,fffff,www.facebook.com,1,1,1.0,2.0,3.0,3.0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,3,8000,4,5,1
6,7,100117,ggggg,www.google.com,1,1,1.0,2.0,NaN,1.0,3,4.0,5.0,6.0,7.0,8.0,9.0,NaN,NaN,NaN,6,5,6000,4,4,1
7,8,100118,hhhhh,www.facebook.com,3,1,2.0,NaN,NaN,2.0,7,8.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,7,5000,5,3,1
8,9,100119,jjjjj,www.youtube.com,1,2,NaN,NaN,NaN,NaN,2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,7000,1,2,2
9,10,100120,kkkkk,www.youtube.com,1,1,1.0,2.0,3.0,1.0,8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,3,6500,3,4,1


Ilustrasi data sebelum kolom multirespon disusun ulang

In [13]:
kolom_labeling = dataset_1h.columns[4:]
kolom_abai = ['q7_harga_bli_rp']

# Melakukan mapping label value
dataset_remap_labeled, _ = rps.labeling_dataset(dataset_1h, label_jawaban, kolom_labeling, kolom_abai)

print("data dengan label sebelum dilakukan re-assign value")
dataset_remap_labeled

data dengan label sebelum dilakukan re-assign value


,nomor,id,kode_alamat,web_fav,kota,q1_puas,q2_alsn_puas_1,q2_alsn_puas_2,q2_alsn_puas_3,q3_alsn_puas_utm,q4_tmpt_bli_1,q4_tmpt_bli_2,q4_tmpt_bli_3,q4_tmpt_bli_4,q4_tmpt_bli_5,q4_tmpt_bli_6,q4_tmpt_bli_7,q4_tmpt_bli_8,q4_tmpt_bli_9,q4_tmpt_bli_10,q5_tmpt_utm,q6_frek_bli,q7_harga_bli_rp,q7_kemurahan,q8_kesan,q9_rekomen
0,1,100111,aaaaa,www.google.com,Jakarta,Tidak,NaN,NaN,NaN,NaN,Pasar tradisional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harga kurang terjangkau,NaN,NaN,Biasa saja,Jelek,Tidak
1,2,100112,bbbbb,www.youtube.com,Aceh,Tidak,NaN,NaN,NaN,NaN,Supermarket,Pasar tradisional,Online shop,Dari keluarga,NaN,NaN,NaN,NaN,NaN,NaN,Sulit digunakan,NaN,NaN,Sangat Mahal,Sangat jelek,Tidak
2,3,100113,ccccc,www.wikipedia.org,Palembang,Ya,Dapat ditemui di mana saja,NaN,NaN,Dapat ditemui di mana saja,Warung,Juragan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tidak banyak dijumpai,NaN,NaN,Mahal,Bagus,Ya
3,4,100114,ddddd,www.youtube.com,Aceh,Tidak,NaN,NaN,NaN,NaN,Warung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tidak banyak dijumpai,NaN,NaN,Mahal,Biasa saja,Tidak
4,5,100115,eeeee,www.youtube.com,Aceh,Ya,Harganya murah,Dapat ditemui di mana saja,NaN,Harganya murah,Warung,Online shop,Dari keluarga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Murah,Sangat bagus,Ya
5,6,100116,fffff,www.facebook.com,Jakarta,Ya,Kualitas baik,Harganya murah,Dapat ditemui di mana saja,Dapat ditemui di mana saja,Juragan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Murah,Sangat bagus,Ya
6,7,100117,ggggg,www.google.com,Jakarta,Ya,Kualitas baik,Harganya murah,NaN,Mudah menggunakannya,Warung,Online shop,Dari keluarga,Dari teman,Mini market,Pom bensin,Toko sembako,NaN,NaN,NaN,NaN,NaN,NaN,Murah,Bagus,Ya
7,8,100118,hhhhh,www.facebook.com,Palembang,Ya,Harganya murah,NaN,NaN,Harganya murah,Mini market,Pom bensin,Toko sembako,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sangat murah,Biasa saja,Ya
8,9,100119,jjjjj,www.youtube.com,Jakarta,Tidak,NaN,NaN,NaN,NaN,Pasar tradisional,Warung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harga kurang terjangkau,NaN,NaN,Sangat Mahal,Jelek,Tidak
9,10,100120,kkkkk,www.youtube.com,Jakarta,Ya,Kualitas baik,Harganya murah,Dapat ditemui di mana saja,Mudah menggunakannya,Pom bensin,Juragan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Biasa saja,Bagus,Ya


Mendapatakan daftar kolom mana saja yang termasuk ke dalam pertanyaan multipleresponse
- first_multi_respon : daftar kolom pertama pada tiap kolom multirespon
- kolom_multi_respon: daftar seluruh kolom multirespon pada dataset


<i> Note : dataset yang digunakan sebagai masukan fungsi <b>get_kolom_multirespon()</b> adalah dataset yang masih bernilai angka-angka saja </i>

In [14]:
first_multi_respon, kolom_multi_respon = rps.get_kolom_multirespon(dataset_1h)

print(first_multi_respon)
print("")
print(kolom_multi_respon)

['q2_alsn_puas_1', 'q4_tmpt_bli_1']

['q2_alsn_puas_1', 'q2_alsn_puas_2', 'q2_alsn_puas_3', 'q4_tmpt_bli_1', 'q4_tmpt_bli_2', 'q4_tmpt_bli_3', 'q4_tmpt_bli_4', 'q4_tmpt_bli_5', 'q4_tmpt_bli_6', 'q4_tmpt_bli_7', 'q4_tmpt_bli_8', 'q4_tmpt_bli_9', 'q4_tmpt_bli_10']


## versi 1

Versi 1 akan menghasilkan kolom multirespon dengan memindahkan jawaban yang sudah ada ke kolom yang sesuai dengan nomor opsi jawaban, kemudian mengubah kodenya dari semula menjadi 1 dengan keterangan label "Ya".

Contoh : semua jawaban di q2 yang bernilai 3 akan dipindahkan ke kolom <b>q2_alsn_puas_opsi_3</b>, kemudian nilainya diubah menjadi 1.

In [15]:
dataset_reassign_v1, new_kolom_multi_respon = rps.reassign_data_multirespon(dataset_1h, first_multi_respon, kolom_multi_respon)

print("data tanpa label setelah dilakukan re-assign value versi 1")
dataset_reassign_v1

data tanpa label setelah dilakukan re-assign value versi 1


,nomor,id,kode_alamat,web_fav,kota,q1_puas,q2_alsn_puas_opsi_1,q2_alsn_puas_opsi_2,q2_alsn_puas_opsi_3,q3_alsn_puas_utm,q4_tmpt_bli_opsi_1,q4_tmpt_bli_opsi_2,q4_tmpt_bli_opsi_3,q4_tmpt_bli_opsi_4,q4_tmpt_bli_opsi_5,q4_tmpt_bli_opsi_6,q4_tmpt_bli_opsi_7,q4_tmpt_bli_opsi_8,q4_tmpt_bli_opsi_9,q4_tmpt_bli_opsi_10,q5_tmpt_utm,q6_frek_bli,q7_harga_bli_rp,q7_kemurahan,q8_kesan,q9_rekomen
0,1,100111,aaaaa,www.google.com,1,2,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,4,5000,3,2,2
1,2,100112,bbbbb,www.youtube.com,2,2,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1,2,7000,1,1,2
2,3,100113,ccccc,www.wikipedia.org,3,1,NaN,NaN,1.0,3.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3,10,6500,2,4,1
3,4,100114,ddddd,www.youtube.com,2,2,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,6000,2,3,2
4,5,100115,eeeee,www.youtube.com,2,1,NaN,1.0,1.0,2.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,4,9,5000,4,5,1
5,6,100116,fffff,www.facebook.com,1,1,1.0,1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,10,3,8000,4,5,1
6,7,100117,ggggg,www.google.com,1,1,1.0,1.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,6,5,6000,4,4,1
7,8,100118,hhhhh,www.facebook.com,3,1,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,7,7,5000,5,3,1
8,9,100119,jjjjj,www.youtube.com,1,2,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,7000,1,2,2
9,10,100120,kkkkk,www.youtube.com,1,1,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,8,3,6500,3,4,1


In [16]:
kolom_labeling = dataset_reassign_v1.columns[4:]
kolom_abai = ['q7_harga_bli_rp']

# Melakukan mapping label value
dataset_reassign_v1_lableled, output_dict_v1 = rps.labeling_dataset(dataset_reassign_v1, label_jawaban, kolom_labeling, kolom_abai,
                                                                    False, new_kolom_multi_respon)

print("data dengan label setelah dilakukan re-assign value versi 1")
dataset_reassign_v1_lableled

data dengan label setelah dilakukan re-assign value versi 1


,nomor,id,kode_alamat,web_fav,kota,q1_puas,q2_alsn_puas_opsi_1,q2_alsn_puas_opsi_2,q2_alsn_puas_opsi_3,q3_alsn_puas_utm,q4_tmpt_bli_opsi_1,q4_tmpt_bli_opsi_2,q4_tmpt_bli_opsi_3,q4_tmpt_bli_opsi_4,q4_tmpt_bli_opsi_5,q4_tmpt_bli_opsi_6,q4_tmpt_bli_opsi_7,q4_tmpt_bli_opsi_8,q4_tmpt_bli_opsi_9,q4_tmpt_bli_opsi_10,q5_tmpt_utm,q6_frek_bli,q7_harga_bli_rp,q7_kemurahan,q8_kesan,q9_rekomen
0,1,100111,aaaaa,www.google.com,Jakarta,Tidak,NaN,NaN,NaN,NaN,NaN,Ya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harga kurang terjangkau,NaN,NaN,Biasa saja,Jelek,Tidak
1,2,100112,bbbbb,www.youtube.com,Aceh,Tidak,NaN,NaN,NaN,NaN,Ya,Ya,NaN,Ya,Ya,NaN,NaN,NaN,NaN,NaN,Sulit digunakan,NaN,NaN,Sangat Mahal,Sangat jelek,Tidak
2,3,100113,ccccc,www.wikipedia.org,Palembang,Ya,NaN,NaN,Ya,Dapat ditemui di mana saja,NaN,NaN,Ya,NaN,NaN,NaN,NaN,NaN,NaN,Ya,Tidak banyak dijumpai,NaN,NaN,Mahal,Bagus,Ya
3,4,100114,ddddd,www.youtube.com,Aceh,Tidak,NaN,NaN,NaN,NaN,NaN,NaN,Ya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tidak banyak dijumpai,NaN,NaN,Mahal,Biasa saja,Tidak
4,5,100115,eeeee,www.youtube.com,Aceh,Ya,NaN,Ya,Ya,Harganya murah,NaN,NaN,Ya,Ya,Ya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Murah,Sangat bagus,Ya
5,6,100116,fffff,www.facebook.com,Jakarta,Ya,Ya,Ya,Ya,Dapat ditemui di mana saja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ya,NaN,NaN,NaN,Murah,Sangat bagus,Ya
6,7,100117,ggggg,www.google.com,Jakarta,Ya,Ya,Ya,NaN,Mudah menggunakannya,NaN,NaN,Ya,Ya,Ya,Ya,Ya,Ya,Ya,NaN,NaN,NaN,NaN,Murah,Bagus,Ya
7,8,100118,hhhhh,www.facebook.com,Palembang,Ya,NaN,Ya,NaN,Harganya murah,NaN,NaN,NaN,NaN,NaN,NaN,Ya,Ya,Ya,NaN,NaN,NaN,NaN,Sangat murah,Biasa saja,Ya
8,9,100119,jjjjj,www.youtube.com,Jakarta,Tidak,NaN,NaN,NaN,NaN,NaN,Ya,Ya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harga kurang terjangkau,NaN,NaN,Sangat Mahal,Jelek,Tidak
9,10,100120,kkkkk,www.youtube.com,Jakarta,Ya,Ya,Ya,Ya,Mudah menggunakannya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ya,NaN,Ya,NaN,NaN,NaN,Biasa saja,Bagus,Ya


In [17]:
# Menyimpan file ke format excel
dataset_reassign_v1.to_excel("dataset reassign/dataset_reassign_v1.xlsx", index=False)

## versi 2

Versi 2 akan menghasilkan kolom multirespon dengan memindahkan jawaban yang sudah ada ke kolom yang sesuai dengan nomor opsi jawaban dengan tetap mempertahankan kodenya data aslinya.

Contoh : semua jawaban di q2 yang bernilai 3 akan dipindahkan ke kolom <b>q2_alsn_puas_opsi_3</b>, dengan nilai tetap 3.

In [18]:
dataset_reassign_v2, new_kolom_multi_respon = rps.reassign_data_multirespon(dataset_1h, first_multi_respon, kolom_multi_respon, versi=2)

print("data tanpa label setelah dilakukan re-assign value versi 2")
dataset_reassign_v2

data tanpa label setelah dilakukan re-assign value versi 2


,nomor,id,kode_alamat,web_fav,kota,q1_puas,q2_alsn_puas_opsi_1,q2_alsn_puas_opsi_2,q2_alsn_puas_opsi_3,q3_alsn_puas_utm,q4_tmpt_bli_opsi_1,q4_tmpt_bli_opsi_2,q4_tmpt_bli_opsi_3,q4_tmpt_bli_opsi_4,q4_tmpt_bli_opsi_5,q4_tmpt_bli_opsi_6,q4_tmpt_bli_opsi_7,q4_tmpt_bli_opsi_8,q4_tmpt_bli_opsi_9,q4_tmpt_bli_opsi_10,q5_tmpt_utm,q6_frek_bli,q7_harga_bli_rp,q7_kemurahan,q8_kesan,q9_rekomen
0,1,100111,aaaaa,www.google.com,1,2,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,4,5000,3,2,2
1,2,100112,bbbbb,www.youtube.com,2,2,NaN,NaN,NaN,NaN,1.0,2.0,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,1,2,7000,1,1,2
2,3,100113,ccccc,www.wikipedia.org,3,1,NaN,NaN,3.0,3.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,3,10,6500,2,4,1
3,4,100114,ddddd,www.youtube.com,2,2,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,6000,2,3,2
4,5,100115,eeeee,www.youtube.com,2,1,NaN,2.0,3.0,2.0,NaN,NaN,3.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,4,9,5000,4,5,1
5,6,100116,fffff,www.facebook.com,1,1,1.0,2.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10,3,8000,4,5,1
6,7,100117,ggggg,www.google.com,1,1,1.0,2.0,NaN,1.0,NaN,NaN,3.0,4.0,5.0,6.0,7.0,8.0,9.0,NaN,6,5,6000,4,4,1
7,8,100118,hhhhh,www.facebook.com,3,1,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,8.0,9.0,NaN,7,7,5000,5,3,1
8,9,100119,jjjjj,www.youtube.com,1,2,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,7000,1,2,2
9,10,100120,kkkkk,www.youtube.com,1,1,1.0,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,10.0,8,3,6500,3,4,1


In [19]:
kolom_labeling = dataset_reassign_v2.columns[4:]
kolom_abai = ['q7_harga_bli_rp']

# Melakukan mapping label value
dataset_reassign_v2_lableled, output_dict_v2 = rps.labeling_dataset(dataset_reassign_v2, label_jawaban, kolom_labeling, kolom_abai,
                                                                    False, new_kolom_multi_respon, versi_1=False)

print("data dengan label setelah dilakukan re-assign value versi 1")
dataset_reassign_v2_lableled

data dengan label setelah dilakukan re-assign value versi 1


,nomor,id,kode_alamat,web_fav,kota,q1_puas,q2_alsn_puas_opsi_1,q2_alsn_puas_opsi_2,q2_alsn_puas_opsi_3,q3_alsn_puas_utm,q4_tmpt_bli_opsi_1,q4_tmpt_bli_opsi_2,q4_tmpt_bli_opsi_3,q4_tmpt_bli_opsi_4,q4_tmpt_bli_opsi_5,q4_tmpt_bli_opsi_6,q4_tmpt_bli_opsi_7,q4_tmpt_bli_opsi_8,q4_tmpt_bli_opsi_9,q4_tmpt_bli_opsi_10,q5_tmpt_utm,q6_frek_bli,q7_harga_bli_rp,q7_kemurahan,q8_kesan,q9_rekomen
0,1,100111,aaaaa,www.google.com,Jakarta,Tidak,NaN,NaN,NaN,NaN,NaN,Pasar tradisional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harga kurang terjangkau,NaN,NaN,Biasa saja,Jelek,Tidak
1,2,100112,bbbbb,www.youtube.com,Aceh,Tidak,NaN,NaN,NaN,NaN,Supermarket,Pasar tradisional,NaN,Online shop,Dari keluarga,NaN,NaN,NaN,NaN,NaN,Sulit digunakan,NaN,NaN,Sangat Mahal,Sangat jelek,Tidak
2,3,100113,ccccc,www.wikipedia.org,Palembang,Ya,NaN,NaN,Dapat ditemui di mana saja,Dapat ditemui di mana saja,NaN,NaN,Warung,NaN,NaN,NaN,NaN,NaN,NaN,Juragan,Tidak banyak dijumpai,NaN,NaN,Mahal,Bagus,Ya
3,4,100114,ddddd,www.youtube.com,Aceh,Tidak,NaN,NaN,NaN,NaN,NaN,NaN,Warung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tidak banyak dijumpai,NaN,NaN,Mahal,Biasa saja,Tidak
4,5,100115,eeeee,www.youtube.com,Aceh,Ya,NaN,Harganya murah,Dapat ditemui di mana saja,Harganya murah,NaN,NaN,Warung,Online shop,Dari keluarga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Murah,Sangat bagus,Ya
5,6,100116,fffff,www.facebook.com,Jakarta,Ya,Kualitas baik,Harganya murah,Dapat ditemui di mana saja,Dapat ditemui di mana saja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juragan,NaN,NaN,NaN,Murah,Sangat bagus,Ya
6,7,100117,ggggg,www.google.com,Jakarta,Ya,Kualitas baik,Harganya murah,NaN,Mudah menggunakannya,NaN,NaN,Warung,Online shop,Dari keluarga,Dari teman,Mini market,Pom bensin,Toko sembako,NaN,NaN,NaN,NaN,Murah,Bagus,Ya
7,8,100118,hhhhh,www.facebook.com,Palembang,Ya,NaN,Harganya murah,NaN,Harganya murah,NaN,NaN,NaN,NaN,NaN,NaN,Mini market,Pom bensin,Toko sembako,NaN,NaN,NaN,NaN,Sangat murah,Biasa saja,Ya
8,9,100119,jjjjj,www.youtube.com,Jakarta,Tidak,NaN,NaN,NaN,NaN,NaN,Pasar tradisional,Warung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harga kurang terjangkau,NaN,NaN,Sangat Mahal,Jelek,Tidak
9,10,100120,kkkkk,www.youtube.com,Jakarta,Ya,Kualitas baik,Harganya murah,Dapat ditemui di mana saja,Mudah menggunakannya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pom bensin,NaN,Juragan,NaN,NaN,NaN,Biasa saja,Bagus,Ya


In [20]:
# Menyimpan file ke format excel
dataset_reassign_v2.to_excel("dataset reassign/dataset_reassign_v2.xlsx", index=False)

# Generate syntax untuk SPSS (Metode 1 membuat File SPSS)

Untuk melakukan generate syntax spss, format data label jawaban yang ada harus ditransformasi dulu menjadi seperti berikut.

In [21]:
label_jawaban_melted = label_jawaban.melt(id_vars='label_jawaban', value_vars=label_jawaban.columns[1:],
                                         var_name='label pertanyaan', value_name='jawaban')

label_jawaban_melted = label_jawaban_melted.dropna()

label_jawaban_melted.head(10)

,label_jawaban,label pertanyaan,jawaban
1,1,kota,Jakarta
2,2,kota,Aceh
3,3,kota,Palembang
12,1,q1,Ya
13,2,q1,Tidak
23,1,q2,Kualitas baik
24,2,q2,Harganya murah
25,3,q2,Dapat ditemui di mana saja
34,1,q3,Mudah menggunakannya
35,2,q3,Harganya murah


## data versi 1

Selain data label jawaban,  dataset yang akan digunakan untuk generate syntax juga perlu ditransformasi dulu menjadi seperti berikut.

In [22]:
data_dipakai_v1 = dataset_reassign_v1.melt(id_vars=dataset_reassign_v1.columns[:4], value_vars=dataset_reassign_v1.columns[4:],
                                           var_name='subpertanyaan', value_name='jawaban')

data_dipakai_v1['label pertanyaan'] = data_dipakai_v1['subpertanyaan'].str.split("_").str[0]  

data_dipakai_v1

,nomor,id,kode_alamat,web_fav,subpertanyaan,jawaban,label pertanyaan
0,1,100111,aaaaa,www.google.com,kota,1.0,kota
1,2,100112,bbbbb,www.youtube.com,kota,2.0,kota
2,3,100113,ccccc,www.wikipedia.org,kota,3.0,kota
3,4,100114,ddddd,www.youtube.com,kota,2.0,kota
4,5,100115,eeeee,www.youtube.com,kota,2.0,kota
...,...,...,...,...,...,...,...
215,6,100116,fffff,www.facebook.com,q9_rekomen,1.0,q9
216,7,100117,ggggg,www.google.com,q9_rekomen,1.0,q9
217,8,100118,hhhhh,www.facebook.com,q9_rekomen,1.0,q9
218,9,100119,jjjjj,www.youtube.com,q9_rekomen,2.0,q9


Selanjutnya, tinggal melakukan pembuatan syntax spss dengan kode program berikut. Secara default, seluruh kolom yang memiliki nama "rp" atau "%" akan di-skip sehingga. 

Note : <b>kolom_abai</b> pada kasus ini dikosongi karena syarat default sudah cukup untuk skip kolom yang tidak ingin di-generate syntax-nya.

In [23]:
nama_file = 'syntax spss/add value label versi 1.txt'
kolom_abai = [] #diisi kolom pertanyaan mana saja yang ingin diskip saat melakukan generate syntax

text_file_syntax = rps.generate_spss_syntax(nama_file=nama_file, data_dipakai=data_dipakai_v1, 
                                            label_jawaban_melted=label_jawaban_melted, 
                                            new_kolom_multi_respon=new_kolom_multi_respon, kolom_abai = kolom_abai, versi=1)

done


In [24]:
file_v1 = open('syntax spss/add value label versi 1.txt', "r")
print(file_v1.read())

add value label kota
1"Jakarta"
2"Aceh"
3"Palembang"

add value label q1_puas
1"Ya"
2"Tidak"

add value label q2_alsn_puas_opsi_1
1"Ya"

add value label q2_alsn_puas_opsi_2
1"Ya"

add value label q2_alsn_puas_opsi_3
1"Ya"

add value label q3_alsn_puas_utm
1"Mudah menggunakannya"
2"Harganya murah"
3"Dapat ditemui di mana saja"

add value label q4_tmpt_bli_opsi_1
1"Ya"

add value label q4_tmpt_bli_opsi_2
1"Ya"

add value label q4_tmpt_bli_opsi_3
1"Ya"

add value label q4_tmpt_bli_opsi_4
1"Ya"

add value label q4_tmpt_bli_opsi_5
1"Ya"

add value label q4_tmpt_bli_opsi_6
1"Ya"

add value label q4_tmpt_bli_opsi_7
1"Ya"

add value label q4_tmpt_bli_opsi_8
1"Ya"

add value label q4_tmpt_bli_opsi_9
1"Ya"

add value label q4_tmpt_bli_opsi_10
1"Ya"

add value label q5_tmpt_utm
1"Sulit digunakan"
2"Harga kurang terjangkau"
3"Tidak banyak dijumpai"

add value label q7_kemurahan
1"Sangat Mahal"
2"Mahal"
3"Biasa saja"
4"Murah"
5"Sangat murah"

add value label q8_kesan
1"Sangat jelek"
2"Jelek"
3"Bias

## data versi 2

In [25]:
data_dipakai_v2 = dataset_reassign_v2.melt(id_vars=dataset_reassign_v2.columns[:4], value_vars=dataset_reassign_v2.columns[4:],
                                           var_name='subpertanyaan', value_name='jawaban')

data_dipakai_v2['label pertanyaan'] = data_dipakai_v2['subpertanyaan'].str.split("_").str[0]  

data_dipakai_v2

,nomor,id,kode_alamat,web_fav,subpertanyaan,jawaban,label pertanyaan
0,1,100111,aaaaa,www.google.com,kota,1.0,kota
1,2,100112,bbbbb,www.youtube.com,kota,2.0,kota
2,3,100113,ccccc,www.wikipedia.org,kota,3.0,kota
3,4,100114,ddddd,www.youtube.com,kota,2.0,kota
4,5,100115,eeeee,www.youtube.com,kota,2.0,kota
...,...,...,...,...,...,...,...
215,6,100116,fffff,www.facebook.com,q9_rekomen,1.0,q9
216,7,100117,ggggg,www.google.com,q9_rekomen,1.0,q9
217,8,100118,hhhhh,www.facebook.com,q9_rekomen,1.0,q9
218,9,100119,jjjjj,www.youtube.com,q9_rekomen,2.0,q9


In [26]:
nama_file = 'syntax spss/add value label versi 2'
kolom_abai = [] #diisi kolom pertanyaan mana saja yang ingin diskip saat melakukan generate syntax

text_file_syntax = rps.generate_spss_syntax(nama_file=nama_file, data_dipakai=data_dipakai_v2, 
                                            label_jawaban_melted=label_jawaban_melted, 
                                            new_kolom_multi_respon=new_kolom_multi_respon, kolom_abai = kolom_abai, versi=2)

done


In [27]:
file_v2 = open('syntax spss/add value label versi 2.txt', "r")
print(file_v2.read())

add value label kota
1"Jakarta"
2"Aceh"
3"Palembang"

add value label q1_puas
1"Ya"
2"Tidak"

add value label q2_alsn_puas_opsi_1
1"Kualitas baik"

add value label q2_alsn_puas_opsi_2
2"Harganya murah"

add value label q2_alsn_puas_opsi_3
3"Dapat ditemui di mana saja"

add value label q3_alsn_puas_utm
1"Mudah menggunakannya"
2"Harganya murah"
3"Dapat ditemui di mana saja"

add value label q4_tmpt_bli_opsi_1
1"Supermarket"

add value label q4_tmpt_bli_opsi_2
2"Pasar tradisional"

add value label q4_tmpt_bli_opsi_3
3"Warung"

add value label q4_tmpt_bli_opsi_4
4"Online shop"

add value label q4_tmpt_bli_opsi_5
5"Dari keluarga"

add value label q4_tmpt_bli_opsi_6
6"Dari teman"

add value label q4_tmpt_bli_opsi_7
7"Mini market"

add value label q4_tmpt_bli_opsi_8
8"Pom bensin"

add value label q4_tmpt_bli_opsi_9
9"Toko sembako"

add value label q4_tmpt_bli_opsi_10
10"Juragan"

add value label q5_tmpt_utm
1"Sulit digunakan"
2"Harga kurang terjangkau"
3"Tidak banyak dijumpai"

add value labe

## Next step

Langkah selanjutnya adalah menyimpan dataset yang sudah dibuat, import data format excel ke SPSS, kemudian melakukan copy-paste syntax yang sudah jadi untuk di-running di SPSS.

# Membuat file SPSS dengan library <b>pyreadstat</b>  (Metode 2 membuat File SPSS)

Langkah ini lebih disarankan jika tidak ingin melakukan running syntax SPSS secara manual. Menggunakan module/library/package <b>pyreadstat</b>, kita dapat langsung membuat file dengan format .sav. Selain itu, library ini juga dapat membaca file .sav di Python.

https://pyreadstat.readthedocs.io/en/latest/ <br>
https://github.com/Roche/pyreadstat <br>


Untuk dapat menulis file ke dalam format .sav, kita perlu membuat sebuah data <b>dictionary</b> yang terdiri dari nama kolom dan label kategorinya.

Data dictionary tersebut sudah terfasilitasi oleh fungsi <b>labeling_dataset()</b> pada modul <b>reassign_into_spss</b>

In [28]:
import pyreadstat

## data versi 1

In [29]:
output_dict_v1

{'kota': {1: 'Jakarta', 2: 'Aceh', 3: 'Palembang'},
 'q1_puas': {1: 'Ya', 2: 'Tidak'},
 'q2_alsn_puas_opsi_1': {1: 'Ya'},
 'q2_alsn_puas_opsi_2': {1: 'Ya'},
 'q2_alsn_puas_opsi_3': {1: 'Ya'},
 'q3_alsn_puas_utm': {1: 'Mudah menggunakannya',
  2: 'Harganya murah',
  3: 'Dapat ditemui di mana saja'},
 'q4_tmpt_bli_opsi_1': {1: 'Ya'},
 'q4_tmpt_bli_opsi_2': {1: 'Ya'},
 'q4_tmpt_bli_opsi_3': {1: 'Ya'},
 'q4_tmpt_bli_opsi_4': {1: 'Ya'},
 'q4_tmpt_bli_opsi_5': {1: 'Ya'},
 'q4_tmpt_bli_opsi_6': {1: 'Ya'},
 'q4_tmpt_bli_opsi_7': {1: 'Ya'},
 'q4_tmpt_bli_opsi_8': {1: 'Ya'},
 'q4_tmpt_bli_opsi_9': {1: 'Ya'},
 'q4_tmpt_bli_opsi_10': {1: 'Ya'},
 'q5_tmpt_utm': {1: 'Sulit digunakan',
  2: 'Harga kurang terjangkau',
  3: 'Tidak banyak dijumpai'},
 'q6_frek_bli': {},
 'q7_harga_bli_rp': {1: 'Sangat Mahal',
  2: 'Mahal',
  3: 'Biasa saja',
  4: 'Murah',
  5: 'Sangat murah'},
 'q7_kemurahan': {1: 'Sangat Mahal',
  2: 'Mahal',
  3: 'Biasa saja',
  4: 'Murah',
  5: 'Sangat murah'},
 'q8_kesan': {1: 'Sang

In [30]:
# Menyimpan file ke format sav (SPSS)
pyreadstat.write_sav(dataset_reassign_v1, "dataset spss/data_spss_versi1.sav", variable_value_labels=output_dict_v1)

## data versi 2

In [31]:
output_dict_v2

{'kota': {1: 'Jakarta', 2: 'Aceh', 3: 'Palembang'},
 'q1_puas': {1: 'Ya', 2: 'Tidak'},
 'q2_alsn_puas_opsi_1': {1: 'Kualitas baik'},
 'q2_alsn_puas_opsi_2': {2: 'Harganya murah'},
 'q2_alsn_puas_opsi_3': {3: 'Dapat ditemui di mana saja'},
 'q3_alsn_puas_utm': {1: 'Mudah menggunakannya',
  2: 'Harganya murah',
  3: 'Dapat ditemui di mana saja'},
 'q4_tmpt_bli_opsi_1': {1: 'Supermarket'},
 'q4_tmpt_bli_opsi_2': {2: 'Pasar tradisional'},
 'q4_tmpt_bli_opsi_3': {3: 'Warung'},
 'q4_tmpt_bli_opsi_4': {4: 'Online shop'},
 'q4_tmpt_bli_opsi_5': {5: 'Dari keluarga'},
 'q4_tmpt_bli_opsi_6': {6: 'Dari teman'},
 'q4_tmpt_bli_opsi_7': {7: 'Mini market'},
 'q4_tmpt_bli_opsi_8': {8: 'Pom bensin'},
 'q4_tmpt_bli_opsi_9': {9: 'Toko sembako'},
 'q4_tmpt_bli_opsi_10': {10: 'Juragan'},
 'q5_tmpt_utm': {1: 'Sulit digunakan',
  2: 'Harga kurang terjangkau',
  3: 'Tidak banyak dijumpai'},
 'q6_frek_bli': {},
 'q7_harga_bli_rp': {1: 'Sangat Mahal',
  2: 'Mahal',
  3: 'Biasa saja',
  4: 'Murah',
  5: 'Sangat m

In [32]:
# Menyimpan file ke format sav (SPSS)
pyreadstat.write_sav(dataset_reassign_v2, "dataset spss/data_spss_versi2.sav", variable_value_labels=output_dict_v2)

## Next step

Selanjutnya, file .sav yang sudah dimiliki sudah siap digunakan di SPSS

Jika suatu saat ingin membuka file tersebut di Python, maka dapat menggunakan kode berikut

In [33]:
print("cara 1")

df, meta = pyreadstat.read_sav('dataset spss/data_spss_versi2.sav')

display(df.head())
print("")
print("dictionary keterangan kolom kategori:")
print(meta.variable_value_labels)

cara 1


,nomor,id,kode_alamat,web_fav,kota,q1_puas,q2_alsn_puas_opsi_1,q2_alsn_puas_opsi_2,q2_alsn_puas_opsi_3,q3_alsn_puas_utm,q4_tmpt_bli_opsi_1,q4_tmpt_bli_opsi_2,q4_tmpt_bli_opsi_3,q4_tmpt_bli_opsi_4,q4_tmpt_bli_opsi_5,q4_tmpt_bli_opsi_6,q4_tmpt_bli_opsi_7,q4_tmpt_bli_opsi_8,q4_tmpt_bli_opsi_9,q4_tmpt_bli_opsi_10,q5_tmpt_utm,q6_frek_bli,q7_harga_bli_rp,q7_kemurahan,q8_kesan,q9_rekomen
0,1.0,100111.0,aaaaa,www.google.com,1.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,5000.0,3.0,2.0,2.0
1,2.0,100112.0,bbbbb,www.youtube.com,2.0,2.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,1.0,2.0,7000.0,1.0,1.0,2.0
2,3.0,100113.0,ccccc,www.wikipedia.org,3.0,1.0,NaN,NaN,3.0,3.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,3.0,10.0,6500.0,2.0,4.0,1.0
3,4.0,100114.0,ddddd,www.youtube.com,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,6000.0,2.0,3.0,2.0
4,5.0,100115.0,eeeee,www.youtube.com,2.0,1.0,NaN,2.0,3.0,2.0,NaN,NaN,3.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,4.0,9.0,5000.0,4.0,5.0,1.0



dictionary keterangan kolom kategori:
{'kota': {1.0: 'Jakarta', 2.0: 'Aceh', 3.0: 'Palembang'}, 'q1_puas': {1.0: 'Ya', 2.0: 'Tidak'}, 'q2_alsn_puas_opsi_1': {1.0: 'Kualitas baik'}, 'q2_alsn_puas_opsi_2': {2.0: 'Harganya murah'}, 'q2_alsn_puas_opsi_3': {3.0: 'Dapat ditemui di mana saja'}, 'q3_alsn_puas_utm': {1.0: 'Mudah menggunakannya', 2.0: 'Harganya murah', 3.0: 'Dapat ditemui di mana saja'}, 'q4_tmpt_bli_opsi_1': {1.0: 'Supermarket'}, 'q4_tmpt_bli_opsi_2': {2.0: 'Pasar tradisional'}, 'q4_tmpt_bli_opsi_3': {3.0: 'Warung'}, 'q4_tmpt_bli_opsi_4': {4.0: 'Online shop'}, 'q4_tmpt_bli_opsi_5': {5.0: 'Dari keluarga'}, 'q4_tmpt_bli_opsi_6': {6.0: 'Dari teman'}, 'q4_tmpt_bli_opsi_7': {7.0: 'Mini market'}, 'q4_tmpt_bli_opsi_8': {8.0: 'Pom bensin'}, 'q4_tmpt_bli_opsi_9': {9.0: 'Toko sembako'}, 'q4_tmpt_bli_opsi_10': {10.0: 'Juragan'}, 'q5_tmpt_utm': {1.0: 'Sulit digunakan', 2.0: 'Harga kurang terjangkau', 3.0: 'Tidak banyak dijumpai'}, 'q7_harga_bli_rp': {1.0: 'Sangat Mahal', 2.0: 'Mahal', 3.0

In [34]:
print("cara 1")

df = pd.read_spss('dataset spss/data_spss_versi2.sav')
df.head()

cara 1


,nomor,id,kode_alamat,web_fav,kota,q1_puas,q2_alsn_puas_opsi_1,q2_alsn_puas_opsi_2,q2_alsn_puas_opsi_3,q3_alsn_puas_utm,q4_tmpt_bli_opsi_1,q4_tmpt_bli_opsi_2,q4_tmpt_bli_opsi_3,q4_tmpt_bli_opsi_4,q4_tmpt_bli_opsi_5,q4_tmpt_bli_opsi_6,q4_tmpt_bli_opsi_7,q4_tmpt_bli_opsi_8,q4_tmpt_bli_opsi_9,q4_tmpt_bli_opsi_10,q5_tmpt_utm,q6_frek_bli,q7_harga_bli_rp,q7_kemurahan,q8_kesan,q9_rekomen
0,1.0,100111.0,aaaaa,www.google.com,Jakarta,Tidak,NaN,NaN,NaN,NaN,NaN,Pasar tradisional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harga kurang terjangkau,4.0,5000.0,Biasa saja,Jelek,Tidak
1,2.0,100112.0,bbbbb,www.youtube.com,Aceh,Tidak,NaN,NaN,NaN,NaN,Supermarket,Pasar tradisional,NaN,Online shop,Dari keluarga,NaN,NaN,NaN,NaN,NaN,Sulit digunakan,2.0,7000.0,Sangat Mahal,Sangat jelek,Tidak
2,3.0,100113.0,ccccc,www.wikipedia.org,Palembang,Ya,NaN,NaN,Dapat ditemui di mana saja,Dapat ditemui di mana saja,NaN,NaN,Warung,NaN,NaN,NaN,NaN,NaN,NaN,Juragan,Tidak banyak dijumpai,10.0,6500.0,Mahal,Bagus,Ya
3,4.0,100114.0,ddddd,www.youtube.com,Aceh,Tidak,NaN,NaN,NaN,NaN,NaN,NaN,Warung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tidak banyak dijumpai,3.0,6000.0,Mahal,Biasa saja,Tidak
4,5.0,100115.0,eeeee,www.youtube.com,Aceh,Ya,NaN,Harganya murah,Dapat ditemui di mana saja,Harganya murah,NaN,NaN,Warung,Online shop,Dari keluarga,NaN,NaN,NaN,NaN,NaN,4.0,9.0,5000.0,Murah,Sangat bagus,Ya


Tidak seperti pyreadstat yang memisahkan tabel dan informasi meta, secara default <b>pandas</b> akan langsung mengubah data kategorik ke label value-nya. Jika tidak ingin mengubahnya, maka perlu mengatur parameter <b>convert_categoricals=False</b>

https://pandas.pydata.org/docs/reference/api/pandas.read_spss.html